In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random

model_name = 'flax-community/papuGaPT2'
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()  

In [18]:
def measure_accuracy(classifier):
	data = []
	for line in open("reviews_for_task3.txt").readlines():
		label, *review = line.strip().split()
		review = " ".join(review)
		data.append(label == classifier(review))
	return sum(data) / len(data)

In [21]:
def classify_review_with_prefix(review):
    pos_sentence = f"Opinia jest pozytywna."
    neg_sentence = f"Opinia jest negatywna."

    pos_prob = sentence_prob(review + " " + pos_sentence)
    neg_prob = sentence_prob(review + " " + neg_sentence)

    if pos_prob > neg_prob:
        return "GOOD"
    else:
        return "BAD"

measure_accuracy(classify_review_with_prefix)

0.7875

In [22]:
def classify_review_with_suffix(review):
    pos_sentence = f"Opinia jest pozytywna."
    neg_sentence = f"Opinia jest negatywna."

    pos_prob = sentence_prob(pos_sentence + " " + review)
    neg_prob = sentence_prob(neg_sentence + " " + review)

    if pos_prob > neg_prob:
        return "GOOD"
    else:
        return "BAD"

measure_accuracy(classify_review_with_suffix)

0.5175